
# QED→Lattice **Phase-locked Manifold Mapper** (v2): Unbroken vs Broken EW Sheets

This version scans the **helical phase** \(\varphi\) under two geometric regimes:

- **Unbroken EW** (pre-Higgs): SU(2) metric sign **negative** (mirror sheet)
- **Broken EW** (post-Higgs): SU(2) metric sign **positive** (physical sheet)

For each regime we enforce **one global normalization** to match \(\alpha_{em}(M_Z)\), then track
\(\sin^2\theta_W\), \(\alpha_s\), and \(\sqrt{\sigma}\).


In [1]:

import math, json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# PDG references
PDG_alpha_MZ    = 1/127.955
PDG_sin2W_MZ    = 0.23122
PDG_alpha_s_MZ  = 0.1179

# Scales, constants
MZ_GEV = 91.1876
HBAR   = 6.582119569e-22  # MeV·s
C      = 299792458.0      # m/s
HBARC  = 0.1973269804     # GeV·fm

# One-loop beta coefficients (MSbar, nf≈5 near MZ)
b1, b2, b3 = 41/10, -19/6, -7

# Bridge inputs
omega_c = 1e23      # Hz
m_Gamma = 17.0      # MeV
chi      = 1.0      # dimensionless

# Derived bridge scales
mu_c_MeV = HBAR * omega_c
MU_C_GEV = mu_c_MeV / 1e3
xi_Gamma = (C / max(omega_c, 1e-30)) * (chi ** (-0.5))

MU_C_GEV, xi_Gamma


(0.06582119569, 2.9979245800000004e-15)

In [2]:

# Running + stiffness dictionary
def run_alpha_to_scale(alpha_mu0, b, mu0, mu):
    if mu <= 0 or mu0 <= 0:
        raise ValueError("Scales must be positive.")
    return 1.0 / ( (1.0/alpha_mu0) - (b/(2*math.pi)) * math.log(mu/mu0) )

# Group-specific exposures (single global normalization later)
EXPO = {
  "U1":  dict(alpha=+0.00, beta=-1.0, gamma=+1/3, p=1.0),
  "SU2": dict(alpha=+0.00, beta=-1.0, gamma=+1/3, p=1.0),
  "SU3": dict(alpha=-0.05, beta=-1.0, gamma=+1/3, p=1.0),  # softened SU(3)
}

def hat_Ki(mu_c_GeV, xi_Gamma, chi, h_i, expo):
    xi_term = (xi_Gamma if xi_Gamma != 0 else 1.0)
    return (h_i**expo["p"]) * ( (xi_term/max(1e-30, 1.0))**expo["beta"] ) * (chi**expo["gamma"]) * (max(mu_c_GeV,1e-12)**expo["alpha"])

def make_barrier_K(mu_c_GeV, xi_Gamma, chi, h_tuple, N=1.0):
    h1,h2,h3 = h_tuple
    K1 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h1, EXPO["U1"])
    K2 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h2, EXPO["SU2"])
    K3 = N * hat_Ki(mu_c_GeV, xi_Gamma, chi, h3, EXPO["SU3"])
    return K1,K2,K3

def run_to_MZ_pair(alpha1_mu, alpha2_mu, MU0_GEV, MZ=MZ_GEV):
    a1_MZ = run_alpha_to_scale(alpha1_mu, b1, MU0_GEV, MZ)
    a2_MZ = run_alpha_to_scale(alpha2_mu, b2, MU0_GEV, MZ)
    return a1_MZ, a2_MZ

def normalize_once_to_alpha_em(K1, K2, MU0_GEV, target_alpha_em=PDG_alpha_MZ):
    def alpha_em_from_scale(s):
        a1_mu0 = (5.0/3.0) / (4*math.pi*(s*K1))   # GUT-normalized U(1)_Y
        a2_mu0 = 1.0        / (4*math.pi*(s*K2))
        A1_MZ, A2_MZ = run_to_MZ_pair(a1_mu0, a2_mu0, MU0_GEV, MZ_GEV)
        return (A1_MZ*A2_MZ) / (A1_MZ + A2_MZ)
    lo, hi = -24.0, +24.0
    flo = alpha_em_from_scale(10**lo)
    fhi = alpha_em_from_scale(10**hi)
    for _ in range(12):
        if flo > target_alpha_em: lo -= 2.0; flo = alpha_em_from_scale(10**lo)
        if fhi < target_alpha_em: hi += 2.0; fhi = alpha_em_from_scale(10**hi)
    for _ in range(96):
        mid = 0.5*(lo+hi)
        fmid = alpha_em_from_scale(10**mid)
        if fmid < target_alpha_em: hi = mid
        else: lo = mid
    return 10**(0.5*(lo+hi))


In [3]:

# Lattice observable + pipeline
C_SIGMA = None

def predict_lattice_observables(K3, xi_Gamma, c_sigma=None, K3_ref=None):
    global C_SIGMA, HBARC
    xi_fm = xi_Gamma / 1e-15
    if xi_fm <= 0: xi_fm = 1e-6
    cS = c_sigma if c_sigma is not None else C_SIGMA
    if cS is None:
        raise RuntimeError("C_SIGMA not set; run calibration.")
    sigma_GeV2 = cS * (K3 / max(xi_fm**2, 1e-30)) * (HBARC**2)
    if K3_ref is None: K3_ref = K3
    a0 = 0.085
    a_fm = a0 * ((K3_ref + 1e-12) / (K3 + 1e-12))**0.10
    a_fm = max(0.05, min(0.14, a_fm))
    return a_fm, sigma_GeV2

def run_full_pipeline(K1, K2, K3, MU0_GEV):
    a1_c = (5.0/3.0) / (4*math.pi*K1)
    a2_c = 1.0        / (4*math.pi*K2)
    a3_c = 1.0        / (4*math.pi*K3)
    a1_MZ = run_alpha_to_scale(a1_c, b1, MU0_GEV, MZ_GEV)
    a2_MZ = run_alpha_to_scale(a2_c, b2, MU0_GEV, MZ_GEV)
    a3_MZ = run_alpha_to_scale(a3_c, b3, MU0_GEV, MZ_GEV)
    alpha_em_MZ = (a1_MZ * a2_MZ) / (a1_MZ + a2_MZ)
    sin2W_MZ    = a1_MZ / (a1_MZ + a2_MZ)
    alpha_s_MZ  = a3_MZ
    a_fm, sigma_GeV2 = predict_lattice_observables(K3, xi_Gamma, K3_ref=K3)
    sqrt_sigma = math.sqrt(abs(sigma_GeV2))
    return {"alpha_em": alpha_em_MZ, "alpha_em_ref": PDG_alpha_MZ,
            "sin2W": sin2W_MZ,       "sin2W_ref":   PDG_sin2W_MZ,
            "alpha_s": alpha_s_MZ,   "alpha_s_ref": PDG_alpha_s_MZ,
            "a_fm": a_fm, "sqrt_sigma": sqrt_sigma}


In [4]:

# Calibrate C_SIGMA once using baseline (1,2,3) on the physical (broken) sheet
K1_b, K2_b, K3_b = make_barrier_K(MU_C_GEV, xi_Gamma, chi, (1.0, 2.0, 3.0), N=1.0)
# enforce positive SU(2) metric for calibration
S_star = normalize_once_to_alpha_em(K1_b, abs(K2_b), MU_C_GEV, PDG_alpha_MZ)
K1_b, K2_b, K3_b = (S_star*K1_b, S_star*abs(K2_b), S_star*K3_b)

xi_fm_b = xi_Gamma / 1e-15
sigma_GeV2_tmp = (K3_b / max(xi_fm_b**2, 1e-30)) * (HBARC**2)
target = 0.44  # GeV
C_SIGMA = (target**2) / max(sigma_GeV2_tmp, 1e-30)
C_SIGMA


3.2920387778245197

In [5]:

# Base ratios and helical parameters (same for both sheets)
r12 = 10/3
r32 = 1/6
eps = {"U1":0.12, "SU2":0.12, "SU3":0.12}
nph = {"U1":1, "SU2":2, "SU3":3}
dph = {"U1":0.0, "SU2":0.0, "SU3":0.0}

def base_K0(sign_SU2=+1.0):
    K1_hat, K2_hat, K3_hat = make_barrier_K(MU_C_GEV, xi_Gamma, chi, (1.0,1.0,1.0), N=1.0)
    K1_0 = K1_hat * r12
    K2_0 = K2_hat / max(r12,1e-12)
    K3_0 = (K2_0 * r32)
    # apply SU(2) metric sign before normalization
    K2_0 = sign_SU2 * K2_0
    S_star0 = normalize_once_to_alpha_em(K1_0, K2_0, MU_C_GEV, PDG_alpha_MZ)
    return S_star0*K1_0, S_star0*K2_0, S_star0*K3_0

def K_phi(K1_0, K2_0, K3_0, phi):
    K1 = K1_0 * (1 + eps["U1"] * math.sin(nph["U1"]*phi + dph["U1"]))
    K2 = K2_0 * (1 + eps["SU2"]* math.sin(nph["SU2"]*phi + dph["SU2"]))
    K3 = K3_0 * (1 + eps["SU3"]* math.sin(nph["SU3"]*phi + dph["SU3"]))
    S = normalize_once_to_alpha_em(K1, K2, MU_C_GEV, PDG_alpha_MZ)
    return S*K1, S*K2, S*K3


In [6]:

# Scan both sheets
outdir = Path("C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/helical_fitting/")
outdir.mkdir(parents=True, exist_ok=True)

def scan_sheet(sheet_name, sign_SU2):
    K1_0, K2_0, K3_0 = base_K0(sign_SU2=sign_SU2)
    Nphi = 720
    phis = np.linspace(0, 2*np.pi, Nphi, endpoint=False)
    residuals = {"sin2W":[], "alpha_s":[], "sqrt_sigma":[], "loss": []}
    tracks    = {"sin2W":[], "alpha_s":[], "sqrt_sigma":[], "alpha_em":[]}
    for phi in phis:
        K1,K2,K3 = K_phi(K1_0, K2_0, K3_0, phi)
        r = run_full_pipeline(K1,K2,K3, MU_C_GEV)
        e1 = abs(r["sin2W"]-r["sin2W_ref"])
        e2 = abs(r["alpha_s"]-r["alpha_s_ref"])
        e3 = abs(r["sqrt_sigma"]-0.44)
        residuals["sin2W"].append(e1)
        residuals["alpha_s"].append(e2)
        residuals["sqrt_sigma"].append(e3)
        residuals["loss"].append(e1+e2+e3)
        tracks["sin2W"].append(r["sin2W"])
        tracks["alpha_s"].append(r["alpha_s"])
        tracks["sqrt_sigma"].append(r["sqrt_sigma"])
        tracks["alpha_em"].append(r["alpha_em"])
    best_idx = int(np.argmin(residuals["loss"]))
    best = {
        "phi": float(phis[best_idx]),
        "loss": float(residuals["loss"][best_idx]),
        "at_best": {
            "sin2W": tracks["sin2W"][best_idx],
            "alpha_s": tracks["alpha_s"][best_idx],
            "sqrt_sigma": tracks["sqrt_sigma"][best_idx],
            "alpha_em": tracks["alpha_em"][best_idx],
        },
        "K0": (K1_0, K2_0, K3_0)
    }
    # Save json and plots for this sheet
    with open(outdir/f"{sheet_name}_summary.json","w") as f:
        json.dump(best, f, indent=2)
    # Plots
    plt.figure()
    plt.plot(phis, residuals["sin2W"], label="|Δ sin²θ_W|")
    plt.plot(phis, residuals["alpha_s"], label="|Δ α_s|")
    plt.plot(phis, residuals["sqrt_sigma"], label="|Δ √σ|")
    plt.plot(phis, residuals["loss"], label="total loss", linewidth=2)
    plt.axvline(best["phi"], linestyle="--")
    plt.xlabel("φ (rad)"); plt.ylabel("residual"); plt.legend(); plt.tight_layout()
    plt.savefig(outdir/f"{sheet_name}_residuals_vs_phi.png", dpi=150); plt.close()

    plt.figure(); plt.plot(phis, tracks["sin2W"]); plt.axhline(PDG_sin2W_MZ, linestyle="--")
    plt.xlabel("φ (rad)"); plt.ylabel("sin²θ_W"); plt.tight_layout()
    plt.savefig(outdir/f"{sheet_name}_track_sin2W.png", dpi=150); plt.close()

    plt.figure(); plt.plot(phis, tracks["alpha_s"]); plt.axhline(PDG_alpha_s_MZ, linestyle="--")
    plt.xlabel("φ (rad)"); plt.ylabel("α_s"); plt.tight_layout()
    plt.savefig(outdir/f"{sheet_name}_track_alpha_s.png", dpi=150); plt.close()

    plt.figure(); plt.plot(phis, tracks["sqrt_sigma"]); plt.axhline(0.44, linestyle="--")
    plt.xlabel("φ (rad)"); plt.ylabel("√σ (GeV)"); plt.tight_layout()
    plt.savefig(outdir/f"{sheet_name}_track_sigma.png", dpi=150); plt.close()

    return best

best_unbroken = scan_sheet("unbroken", sign_SU2=-1.0)
best_broken   = scan_sheet("broken",   sign_SU2=+1.0)

best_unbroken, best_broken


({'phi': 0.3490658503988659,
  'loss': 4.070262174584978,
  'at_best': {'sin2W': -3.3928571428571423,
   'alpha_s': 0.12408503172790528,
   'sqrt_sigma': 6.895532492275553e-14,
   'alpha_em': -0.9306377379592896},
  'K0': (20.133027272088093, -1.8119724544879283, 0.30199540908132133)},
 {'phi': 0.3141592653589793,
  'loss': 4.070262174584988,
  'at_best': {'sin2W': -3.3928571428571423,
   'alpha_s': 0.12408503172790528,
   'sqrt_sigma': 5.909877841719096e-14,
   'alpha_em': -0.9306377379592896},
  'K0': (14.880933201108652, 1.3392839880997784, 0.22321399801662975)})

In [7]:

# Overlay comparison plot (loss vs φ) for quick visual
Nphi = 720
phis = np.linspace(0, 2*np.pi, Nphi, endpoint=False)

def load_track(fname):
    import json
    with open(fname) as f:
        return json.load(f)

import json
from pathlib import Path
import numpy as np

# Use the same output directory defined in the previous cell
outdir = Path("/mnt/data/experiments/qed_to_lattice/phase_scan_v2")

# Construct the correct file paths using the outdir variable
unbroken_path = outdir / "unbroken_summary.json"
broken_path   = outdir / "broken_summary.json"

# Load the data from the correct paths
with open(unbroken_path, 'r') as f:
    bu = json.load(f)

with open(broken_path, 'r') as f:
    bb = json.load(f)

# Access the keys directly from the dictionaries
print("Unbroken (mirror) at φ*:", bu["phi"], "loss:", bu["loss"], bu["at_best"])
print("Broken (physical) at φ*:", bb["phi"], "loss:", bb["loss"], bb["at_best"])


Unbroken (mirror) at φ*: 0.3490658503988659 loss: 4.070262174584978 {'sin2W': -3.3928571428571423, 'alpha_s': 0.12408503172790528, 'sqrt_sigma': 6.895532492275553e-14, 'alpha_em': -0.9306377379592896}
Broken (physical) at φ*: 0.3141592653589793 loss: 4.070262174584988 {'sin2W': -3.3928571428571423, 'alpha_s': 0.12408503172790528, 'sqrt_sigma': 5.909877841719096e-14, 'alpha_em': -0.9306377379592896}
